In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

import subprocess
import os.path
from time import time

import uproot

In [ ]:
PERIOD = 'LHC18f'
MAIN_DIR = '/alice/data/2018/'
FNAME = 'FilterEvents_Trees.root'
BRANCHES = ['runNumber', 'evtTimeStamp', 'v0.fEffMass']

TIME_OFFSET = 1262307600 - 7200

# IN ROOT

In [ ]:
os.listdir('data_validation_V0s/')

In [ ]:
%%time 

def root2df(fname):
    return uproot.open(fname)['v0_mass'].pandas.df()

V0s = [root2df('data_validation_V0s/'+f) for f in os.listdir('data_validation_V0s/') if PERIOD in f]
print(len(V0s))

In [ ]:
V0 = pd.concat(V0s)
del V0s
V0.describe()

In [ ]:
df = pd.read_csv('data/trending_merged_{}_withGraphs.csv'.format(PERIOD))
# v0 = pd.read_csv('data_validation_V0s/validation_V0s_LHC18p_000294529.csv')
# df = df.query('run == 294529')
print(len(df))
df.head()

In [ ]:
sorted(list(set(df['run'])))

In [ ]:
runx = 287977
print('min:', V0.query('abs(runNumber - @runx) < 1e-6')['evtTimeStamp'].min() - TIME_OFFSET)
print('max:', V0.query('abs(runNumber - @runx) < 1e-6')['evtTimeStamp'].max() - TIME_OFFSET)
print()
print('min chunkStart', df.query('abs(run - @runx) < 1e-6')['chunkStart'].min())
print('max chunkStop', df.query('abs(run - @runx) < 1e-6')['chunkStop'].max())

In [ ]:
mini = df.query('abs(run - @runx) < 1e-6')['chunkStart'].min()
maxi = df.query('abs(run - @runx) < 1e-6')['chunkStop'].max()
fig = plt.figure(figsize=(20,5))
plt.hist(V0.query('abs(runNumber - @runx) < 1e-6')['evtTimeStamp'] - TIME_OFFSET, bins=50, histtype='step', density=1);
plt.vlines(x=[mini,maxi], ymin=0,ymax=plt.ylim()[1])

In [ ]:
plt.figure(figsize=(250,4))
plt.hist(V0['evtTimeStamp'].to_numpy()-TIME_OFFSET, bins=500, density=1, histtype='step');
ymax = plt.ylim()[1] 
for start, stop in df[['chunkStart', 'chunkStop']].to_numpy():
    plt.hlines(ymax - (np.random.rand())*0.1*ymax, xmin=start, xmax=stop)

In [ ]:
for runx in sorted(list(set(df['run']))):
    chunk_min =  df.query('run == @runx')['chunkStart'].min()
    chunk_max = df.query('run == @runx')['chunkStop'].max()
    fname = 'data_validation_V0s/validation_V0s_{}_000{}.csv'.format(PERIOD,runx)
    if not os.path.isfile(fname):
        continue
    v0 = pd.read_csv(fname)['evtTimeStamp'] - TIME_OFFSET
    ts_min = min(v0)
    ts_max = max(v0)
    print('{run}: chunks: {chunk_min} - {chunk_max}\t timeStamps: {ts_min} - {ts_max}'.format(run=runx, chunk_min=chunk_min, chunk_max=chunk_max, ts_min=ts_min, ts_max=ts_max))

In [ ]:
V0.shape

In [ ]:
vals =  df['chunkMean'].tolist()+ (V0['evtTimeStamp']-TIME_OFFSET).tolist()
bins = np.linspace(min(vals), max(vals), 150)
plt.figure(figsize=(15,5))
plt.hist(df['chunkMean'], histtype='step', bins=bins, color='b', density=1);
plt.hist(V0['evtTimeStamp'] - TIME_OFFSET, histtype='step', bins=bins, color='orange', density=1);

In [ ]:
%%time

hist_lst = []
bad_lst = []
mean_lst = []
std_lst = []
num_lst = []
for i,row in enumerate(df[['chunkStart', 'chunkStop', 'alias_global_Warning']].to_numpy()):
    lo, hi, bad = row
    mass_arr = V0.query('evtTimeStamp - @TIME_OFFSET > {lo} & evtTimeStamp - @TIME_OFFSET < {hi}'.format(lo=lo,hi=hi, TIME_OFFSET=TIME_OFFSET))['v0.fEffMass']
    flag = '\t\t\t<<----------' if bad else '' 

    mass_arr_peak = []
    mmin, mmax = 0.475, 0.52
    for m in mass_arr:
        if m > mmin and m < mmax: mass_arr_peak.append(m)

    if len(mass_arr_peak) > 0:
        bad_lst.append(bad)
        mean_lst.append(np.mean(mass_arr_peak))
        std_lst.append(np.std(mass_arr_peak))
        num_lst.append(len(mass_arr_peak))
        
    print('{:3d}: n = {:<8d} {:.4f} +/- {:.4f} {}'.format(i,len(mass_arr_peak), np.mean(mass_arr_peak), np.std(mass_arr_peak), flag))

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
for bad in [0,1]:
    x = [m for m,b in zip(mean_lst, bad_lst) if b == bad]
    y = [s for s,b in zip(std_lst , bad_lst) if b == bad]
    c = 'r' if bad else 'b'
    alpha = 0.8 if bad else 0.2
    s = 60 if bad else 30
    ax.scatter(x,y, color=c, s=s, edgecolors='k', alpha=alpha)
    

ax.set_xlim(0.497, 0.499);
ax.set_ylim(0.008, 0.010);

ax.set_xlabel('mass aver. [MeV]')
ax.set_ylabel('mass RMS [MeV]')
# plt.savefig(f'graphics/V0_results/V0_valid_{PERIOD}.png')